In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn import metrics
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import MinMaxScaler
from env import *
from wrangle import *
import socket

In [2]:
df = wrangle_logs()

In [3]:
df.ip_name.value_counts()

097-105-019-058.biz.spectrum.com                 284579
097-105-019-061.biz.spectrum.com                  61662
71-150-217-33.lightspeed.rcsntx.sbcglobal.net      6791
cpe-76-185-145-231.satx.res.rr.com                 4754
host194.regusus.com                                4360
                                                  ...  
rrcs-71-40-74-50.sw.biz.rr.com                        1
104-184-27-54.lightspeed.snantx.sbcglobal.net         1
cpe-76-182-166-199.sc.res.rr.com                      1
209-107-210-228.ord.as54203.net                       1
072-182-113-134.res.spectrum.com                      1
Name: ip_name, Length: 3817, dtype: int64

In [4]:
df.shape

(900223, 16)

# 3. Are there students who, when active, hardly access the curriculum? If so, what information do you have about these students?

In [11]:
df.user_id.value_counts().tail(10)

163    1
649    1
652    1
165    1
940    1
66     1
879    1
952    1
918    1
593    1
Name: user_id, dtype: int64

# 4. Is there any suspicious activity, such as users/machines/etc accessing the curriculum who shouldn’t be? Does it appear that any web-scraping is happening? Are there any suspicious IP addresses?

In [8]:
df[df.path.isna() == True]

,time,path,user_id,cohort_id,ip,id,name,slack,start_date,end_date,created_at,updated_at,deleted_at,program_id,datetime,ip_name
date,,,,,,,,,,,,,,,,
2020-04-08,09:25:18,NaN,586,55.0,72.177.240.51,55.0,Curie,#curie,2020-02-03,2020-07-07,2020-02-03 19:31:51,2020-02-03 19:31:51,NaN,3.0,2020-04-08 09:25:18,072-177-240-051.res.spectrum.com


In [12]:
df = df.drop(columns='id')

,time,path,user_id,cohort_id,ip,name,slack,start_date,end_date,created_at,updated_at,deleted_at,program_id,datetime,ip_name
date,,,,,,,,,,,,,,,
2018-01-26,09:55:03,/,1,8.0,97.105.19.61,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1.0,2018-01-26 09:55:03,097-105-019-061.biz.spectrum.com
2018-01-26,09:56:02,java-ii,1,8.0,97.105.19.61,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1.0,2018-01-26 09:56:02,097-105-019-061.biz.spectrum.com
2018-01-26,09:56:05,java-ii/object-oriented-programming,1,8.0,97.105.19.61,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1.0,2018-01-26 09:56:05,097-105-019-061.biz.spectrum.com
2018-01-26,09:56:06,slides/object_oriented_programming,1,8.0,97.105.19.61,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1.0,2018-01-26 09:56:06,097-105-019-061.biz.spectrum.com
2018-01-26,09:56:24,javascript-i/conditionals,2,22.0,97.105.19.61,Teddy,#teddy,2018-01-08,2018-05-17,2018-01-08 13:59:10,2018-01-08 13:59:10,NaN,2.0,2018-01-26 09:56:24,097-105-019-061.biz.spectrum.com
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-04-21,14:41:08,toc,939,138.0,174.197.2.198,Neptune,#neptune,2021-03-15,2021-09-03,2021-03-15 19:57:09,2021-03-15 19:57:09,NaN,2.0,2021-04-21 14:41:08,198.sub-174-197-2.myvzw.com
2021-04-21,14:41:11,html-css/css-ii/bootstrap-grid-system,939,138.0,174.197.2.198,Neptune,#neptune,2021-03-15,2021-09-03,2021-03-15 19:57:09,2021-03-15 19:57:09,NaN,2.0,2021-04-21 14:41:11,198.sub-174-197-2.myvzw.com
2021-04-21,15:41:45,/,887,135.0,107.77.169.13,Marco,#marco,2021-01-25,2021-07-19,2021-01-20 21:31:11,2021-01-20 21:31:11,NaN,2.0,2021-04-21 15:41:45,mobile-107-77-169-13.mobile.att.net
